In [12]:
import ast
import os
import pandas as pd

data_names = ['bretschneider-th-main', 'bretschneider-th-school', 'cmsb-tsd', 'gao-2018-fhc', 'gibert-2018-shs', 'twitter-hate-speech-tsa', 'us-election-2020', 'waseem-and-hovy-2016']
multi_class_data_names = ['ami', 'davidson-thon', 'founta-2018-thas']
data_cols = ['data_name', 'num_classes', 'data_type', 'label_col',]
variant_cols = ['variant', 'sampling_modifiedRS_rho', 'sampling_weightedRS_percentage', 'loss', 'wce_alpha', 'fl_gamma',]
metrics_cols = ['val_f1_macro',
                'test_f1_macro', 
                'test_f1_per_label_0', 
                'test_f1_per_label_1',
                'test_f1_per_label_2', 
                'test_f1_per_label_3',
                'test_f1_per_label_4',
                'test_accuracy', 
                'test_precision_macro',
                'test_precision_weighted',
                'test_recall_macro',
                'test_recall_weighted',
                'test_auprc',]
COLS = data_cols + variant_cols + ['pl_seed'] + metrics_cols + ['mlflow_run_id']

results_excel_path = "results_all.xlsx"
cols_results = ['data_name', 'num_classes', 
                'variant', 'sampling_modifiedRS_rho', 'sampling_weightedRS_percentage', 'wce_alpha', 'fl_gamma',
                'val_f1_macro', 'test_f1_macro', 'test_f1_macro_std'] + metrics_cols[2:]

In [15]:
def get_value_by_run_id(data_dir, run_id, key):
    value = None
    for root, _, files in os.walk(data_dir + run_id):  
        if key in files:
            with open(f"{root}/{key}", "r") as f:
                value = f.readlines()[-1]
                if "val" in key or "test" in key:
                    value = value.split()[1]
                if value == "sampling_modifiedRS": # Forgot to specify in the variant value
                    value += "_oversampling"
                try:
                    value = ast.literal_eval(value)
                    if isinstance(value, list):
                        value = [round(v, 2) for v in value]
                        value = tuple(value)
                except (ValueError, SyntaxError):
                    pass
                break
    if value is None:
        value = "-"
        if "test" in key:
            value = 0
    return value

def get_log_by_data_name(data_name):
    data_runs_dir = f'../logs/{data_name}/'
    run_ids = os.listdir(data_runs_dir)
    rows = []
    for run_id in run_ids:
        run_result = {col_name: get_value_by_run_id(data_runs_dir, run_id, col_name) for col_name in COLS[:-1]}
        run_result['mlflow_run_id'] = run_id
        rows.append(run_result)
    df = pd.DataFrame(columns=COLS, data=rows)
    df.to_csv(f"results_{data_name}_raw.csv", index=False)
    return df

def aggregate_results(data_name, writer):
    # Get raw mlflow logs:
    df = get_log_by_data_name(data_name)
    df.to_excel(writer, sheet_name=f"{data_name}_raw")
    assert len(df) % 3 == 0
    # assert len(df[df["test_f1_macro"] == 0]) == 0
    # Aggregate results from 3 seeds
    agg_target = {metric: 'mean' for metric in metrics_cols}
    agg_target.update({col: 'first' for col in data_cols})
    df['test_f1_macro_std'] = df.loc[:, 'test_f1_macro']
    agg_target.update({'test_f1_macro_std': 'std'})
    df_seeds_mean = df.groupby(variant_cols, as_index=False).agg(agg_target)
    #  - add wfl(alpha=1.0) => fl
    df_seeds_mean.loc[(df_seeds_mean["variant"] == "wfl") & (df_seeds_mean["wce_alpha"] == 1.0),  "variant"] = "fl"
    #  - change metric to be 00.00
    for metric in metrics_cols + ['test_f1_macro_std']:
        df_seeds_mean[metric] = df_seeds_mean[metric].apply(lambda x: "{0:.2f}".format(x*100))
    #  - re-order the columns
    df_seeds_mean = df_seeds_mean[cols_results]
    df_seeds_mean.to_excel(writer, sheet_name=f"{data_name}_seeds-mean")
    df_val_f1_max = df_seeds_mean.loc[df_seeds_mean.groupby(["variant"])['val_f1_macro'].idxmax()].reset_index(drop=True)
    return df_val_f1_max

In [16]:
writer = pd.ExcelWriter(results_excel_path)
df_all_agg_results = pd.DataFrame(columns=cols_results)
for data_name in data_names + multi_class_data_names:
    df_val_f1_max = aggregate_results(data_name, writer)
    df_all_agg_results = pd.concat([df_all_agg_results, df_val_f1_max], ignore_index=True)
df_all_agg_results = df_all_agg_results.sort_values(by=['data_name']).reset_index(drop=True)
df_all_agg_results.to_excel(writer, sheet_name=f"best_of_all")
writer.close()

/mounts/Users/cisintern/zhangyaq/anaconda3/envs/pl_env/lib/python3.12/site-packages/openpyxl/workbook/child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
/mounts/Users/cisintern/zhangyaq/anaconda3/envs/pl_env/lib/python3.12/site-packages/openpyxl/workbook/child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
/mounts/Users/cisintern/zhangyaq/anaconda3/envs/pl_env/lib/python3.12/site-packages/openpyxl/workbook/child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


In [17]:
df_all_agg_results[["data_name", "variant", "test_f1_macro"]]

,data_name,variant,test_f1_macro
0,bretschneider-th-main,baseline,68.89
1,bretschneider-th-main,sampling_modifiedRS_oversampling,73.72
2,bretschneider-th-main,sampling_weightedRS,70.68
3,bretschneider-th-main,th,70.60
4,bretschneider-th-main,wce,70.82
5,bretschneider-th-school,wce,74.49
6,bretschneider-th-school,sampling_weightedRS,73.99
7,bretschneider-th-school,th,74.11
8,bretschneider-th-school,baseline,69.82
9,bretschneider-th-school,sampling_modifiedRS_oversampling,76.23


In [6]:
mlrun_ids = []

with open("../outputs/bretschneider-th-school_sampling_weightedRS_seed0_output.txt", "r") as f:
    log_contents = f.readlines()
    mlrun_ids += [log.split("/")[-1][:-2] for log in log_contents if "MLflow Saved Child Search" in log]

print(mlrun_ids)

['591e1b6a525c4c1dba299078504b6c90', '2a8795a8ef444002bba32c9593ddeafd', '6ae407f7e7554429b704e521e2efb632', 'b4067e94e0b24f5dbaa2337056dc7578', '442c82a0d1d845098b81eba8565ba978']


In [8]:
data_runs_dir = f'../logs/us-election-2020/'
run_ids = os.listdir(data_runs_dir)
logs = dict()
for run_id in run_ids:
    ckpt_dir = f"{data_runs_dir}{run_id}/artifacts/model_checkpoints/"
    try:
        ckpt = os.listdir(ckpt_dir)[0]
        if "wce_alpha=0.5" in ckpt:    
            logs[run_id] = ckpt
            print(f"{run_id}\t{ckpt}")
    except NotADirectoryError:
        pass
len(logs)

3fa9dacc654d4dd993daa99f3154797d	us-election-2020-wfl-Trial_32-wce_alpha=0.5-fl_gamma=0.2-seed0-epoch=04-val_f1_macro=0.81.ckpt
059235d3daee43e8abb12b633429386e	us-election-2020-wfl-Trial_22-wce_alpha=0.5-fl_gamma=2.0-seed42-epoch=09-val_f1_macro=0.79.ckpt
812856c9519d41a294ca58b95df133a0	us-election-2020-wfl-Trial_1-wce_alpha=0.5-fl_gamma=1.0-seed42-epoch=05-val_f1_macro=0.81.ckpt
d0011460968d440ea1f49694f9c2054a	us-election-2020-wfl-Trial_17-wce_alpha=0.5-fl_gamma=0.1-seed42-epoch=03-val_f1_macro=0.79.ckpt
37dfc6631af44732a68131dbe49eaa3c	us-election-2020-wfl-Trial_21-wce_alpha=0.5-fl_gamma=0.2-seed21-epoch=08-val_f1_macro=0.79.ckpt
3e4fbd90dc3f4d95ba4bac5e6ed52864	us-election-2020-wfl-Trial_35-wce_alpha=0.5-fl_gamma=2.0-seed0-epoch=06-val_f1_macro=0.78.ckpt
9ec9ef85e7414db9bfeb9d1157e0b502	us-election-2020-wfl-Trial_26-wce_alpha=0.5-fl_gamma=0.2-seed42-epoch=03-val_f1_macro=0.81.ckpt
ba14e53c72b748b78f2386dee1bc74a1	us-election-2020-wfl-Trial_8-wce_alpha=0.5-fl_gamma=2.0-seed21-epoc

18